# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import myKey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)  
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
#uniform is to have the numbers evenly distributed
#size is how many random numbers will be generated.
#we are saying, generate 1500 random evenly distributed numbers between -90 and 90.

# Identify nearest city for each lat, lng combination
for each in lat_lngs:
    city = citipy.nearest_city(each[0], each[1]).city_name 
    country = citipy.nearest_city(each[0], each[1]).country_code 
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
len(cities)

#612 cites are returned.   There must be at least 500 on the next part of finding the weather.  If not, then must come back to this step and increase size.

626

In [3]:
len(countries)

626

In [4]:
#??  how do I see the ln_lngs list?
print(lat_lngs)

In [5]:
print(lngs)

[-168.11595967  128.34053466   89.99244618 ... -155.72636944 -131.97150403
 -155.41412964]


In [6]:
#

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
#first, perform a check on a single city.

In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "alofi"
units = "imperial"

# Build query URL
query_url = url + "appid=" + myKey + "&q=" + city + "&units=imperial"
#note:   adding imperial units will return the temperature in farenheight and the wind speed in miles per hour
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&q=alofi&units=imperial


In [7]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()


In [8]:
print(json.dumps(weather_json, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 44
    },
    "cod": 200,
    "coord": {
        "lat": -19.06,
        "lon": -169.92
    },
    "dt": 1554051600,
    "id": 4036284,
    "main": {
        "humidity": 100,
        "pressure": 1009,
        "temp": 75.2,
        "temp_max": 75.2,
        "temp_min": 75.2
    },
    "name": "Alofi",
    "rain": {
        "3h": 0.142
    },
    "sys": {
        "country": "NU",
        "id": 7306,
        "message": 0.0047,
        "sunrise": 1554053160,
        "sunset": 1554096080,
        "type": 1
    },
    "weather": [
        {
            "description": "light rain",
            "icon": "10n",
            "id": 500,
            "main": "Rain"
        }
    ],
    "wind": {
        "deg": 230,
        "speed": 2.24
    }
}


In [9]:
#Success!   Able to retrieve a single city.
#now retrieve each individual piece of data needed for the loop

In [10]:
print(weather_json["name"])

Alofi


In [11]:
print(weather_json["coord"]["lat"])

-19.06


In [12]:
print(weather_json["main"]["temp"])

75.2


In [13]:
print(weather_json["main"]["humidity"])

100


In [14]:
print(weather_json["clouds"]["all"])

44


In [15]:
print(weather_json["wind"]["speed"])

2.24


### need a dataframe containing city and country to begin
NOTE:   the Weather API will take only a city; however, if multiples are found, API responds with a list of results that match a searching word. 
<BR>
Therefore, I will begin with a list of cities and countries which can also be passed to the 
    <BR>q city name and country code divided by comma, use ISO 3166 country codes
        
<BR> https://openweathermap.org/current

In [16]:
#Begin with the city and country list which was created in the begining.

cityCountry = pd.DataFrame(list(zip(cities, countries)),
              columns=['city','country'])

cityCountry.head()

,city,country
0,vaini,to
1,tiksi,ru
2,dikson,ru
3,sibu,my
4,faanui,pf


In [17]:
#confirm there are at least 500 countries
len(cityCountry)

626

In [18]:
#  ??? not sure if I will take this approach

#the weather API allows for no more than 60 calls per minutes.
#divide the list into groups of 60
#https://openweathermap.org/price

# Add column for batch
# Note that we used "" to specify initial entry.
cityCountry["batch"] = ""
cityCountry.head()

,city,country,batch
0,vaini,to,
1,tiksi,ru,
2,dikson,ru,
3,sibu,my,
4,faanui,pf,


In [19]:
# create a data frome with the values needed for the API calls

cityCountry2 = cityCountry
cityCountry2["location"] = (cityCountry2["city"] + ", " + cityCountry2["country"])
cityCountry2["newCity"] = ""
cityCountry2["newCountry"] = ""
cityCountry2["lat"] = ""
cityCountry2["long"] = ""
cityCountry2["tempF"] = ""
cityCountry2["humidity"] = ""
cityCountry2["cloud"] = ""
cityCountry2["windMPH"] = ""
cityCountry2["maxTemp"] = ""
cityCountry2.head()

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,vaini,to,,"vaini, to",,,,,,,,,
1,tiksi,ru,,"tiksi, ru",,,,,,,,,
2,dikson,ru,,"dikson, ru",,,,,,,,,
3,sibu,my,,"sibu, my",,,,,,,,,
4,faanui,pf,,"faanui, pf",,,,,,,,,


### Populate the data only for a few records
Practice before attemping to get data for 500+ cities

In [20]:
#create a new df with only 10 records
cityCountrySample = cityCountry2.head(10)
cityCountrySample.head(20)

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,vaini,to,,"vaini, to",,,,,,,,,
1,tiksi,ru,,"tiksi, ru",,,,,,,,,
2,dikson,ru,,"dikson, ru",,,,,,,,,
3,sibu,my,,"sibu, my",,,,,,,,,
4,faanui,pf,,"faanui, pf",,,,,,,,,
5,chanderi,in,,"chanderi, in",,,,,,,,,
6,kruisfontein,za,,"kruisfontein, za",,,,,,,,,
7,east london,za,,"east london, za",,,,,,,,,
8,dunmore town,bs,,"dunmore town, bs",,,,,,,,,
9,burica,pa,,"burica, pa",,,,,,,,,


In [38]:
#ensure I can get a single response using parameters
parameters = {"appid": myKey,
             "units": "imperial",
             "q": "vaini, to"}
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [43]:
#make the response in json format by adding .json()
singleResponse = requests.get(base_url, params=parameters)

In [44]:
print(singleResponse.url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=vaini%2C+to


In [45]:
singleResponseJ = singleResponse.json()

In [46]:
print(json.dumps(singleResponseJ, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 20
    },
    "cod": 200,
    "coord": {
        "lat": -21.2,
        "lon": -175.2
    },
    "dt": 1554051600,
    "id": 4032243,
    "main": {
        "humidity": 100,
        "pressure": 1010,
        "temp": 71.6,
        "temp_max": 71.6,
        "temp_min": 71.6
    },
    "name": "Vaini",
    "sys": {
        "country": "TO",
        "id": 7285,
        "message": 0.004,
        "sunrise": 1554140885,
        "sunset": 1554183652,
        "type": 1
    },
    "visibility": 10000,
    "weather": [
        {
            "description": "few clouds",
            "icon": "02n",
            "id": 801,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 210,
        "speed": 3.36
    }
}


In [47]:
print(singleResponseJ["name"])

Vaini


In [ ]:
#success!  I can get a valid URL using parameters.  

In [50]:
# create a parameters dict that will be updated with new city each iteration
parameters = {"appid": myKey,
             "units": "imperial"}

# Loop through the cityCountrySample and perform a search on each
for index, row in cityCountrySample.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #confirm the URL
    print(cities_data.url)
        
    # convert to json
    citiesJ = cities_data.json()
    
    #confirm the data in the response
    print(citiesJ["name"])
    
    try:
    cityCountrySample.loc[index, "newCity"] = citiesJ["name"]
    cityCountrySample.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    
    except:
    print("Missing field/result... skipping.")  

# Visualize to confirm new city and new country appear
cityCountrySample.head()


http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=vaini%2Cto
Vaini


C:\Users\lorig\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lorig\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=tiksi%2Cru
Tiksi
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=dikson%2Cru
Dikson
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=sibu%2Cmy
Sibu
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=faanui%2Cpf
Faanui
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=chanderi%2Cin
Chanderi
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=kruisfontein%2Cza
Kruisfontein
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=east+london%2Cza
East London
http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=dunmore+town%2Cbs
Dunmo

KeyError: 'name'

In [52]:
cityCountrySample.head(10)

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,vaini,to,,"vaini, to",Vaini,TO,,,,,,,
1,tiksi,ru,,"tiksi, ru",Tiksi,RU,,,,,,,
2,dikson,ru,,"dikson, ru",Dikson,RU,,,,,,,
3,sibu,my,,"sibu, my",Sibu,MY,,,,,,,
4,faanui,pf,,"faanui, pf",Faanui,PF,,,,,,,
5,chanderi,in,,"chanderi, in",Chanderi,IN,,,,,,,
6,kruisfontein,za,,"kruisfontein, za",Kruisfontein,ZA,,,,,,,
7,east london,za,,"east london, za",East London,ZA,,,,,,,
8,dunmore town,bs,,"dunmore town, bs",Dunmore Town,BS,,,,,,,
9,burica,pa,,"burica, pa",,,,,,,,,


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot